In [12]:
from pdfminer.high_level import extract_text

import pandas as pd
from copy import copy

In [ ]:
import docx2pdf

docx2pdf.convert("../data/files/РыбаковСергейВячеславович_Supl.docx", '../data/files/converted_from_docx.pdf')

In [13]:
def rus(text, alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    return not alphabet.isdisjoint(text.lower())

In [14]:
def eng(text, alphabet=set('abcdefghijklmnopqrstuvwxyz')):
    return not alphabet.isdisjoint(text.lower())

In [15]:
def find_extract(text: str, text_to_find: str, bias_=0, without_new_str=False, until=None):

    idx = text.find(text_to_find) + len(text_to_find) + bias_

    list_ = []

    if until is None:
        while not(eng(text[idx])):
            if text[idx] != '\n':
                list_.append(text[idx])
            else:
                if without_new_str:
                    break
            idx += 1
    else:
        while not(text[idx:].find(until) == 0):
            list_.append(text[idx])
            idx += 1

    return(''.join(list_[:-1]))

In [16]:
def parse_marks(text):

    idx = 0
    list_ = []

    for i in range(len(text)):
        if rus(text[i]) or text[i] == ' ':
            list_.append(text[i])

    list_ = ''.join(list_).split(' ')

    return [x for x in list_ if x in ('удовлетворительно', 'хорошо', 'отлично', 'зачтено')]

In [17]:
def parse_diploma_supplement(pdf_path):

    pdf_file = open(pdf_path, 'rb')
    text = extract_text(pdf_file, password='pass', page_numbers=None, maxpages=0, caching=True, codec='utf-8', laparams=None)

    general_info = pd.DataFrame([], columns=['family_name', 'date_of_birth', 'access_requirements', 'admitted_to', 'graduated_from', 'level_of_education', 'name_of_qualification', 'main_field_of_study', 'academic_programme_title', 'professional_trajectory',
                                'official_length_of_programme', 'mode_of_study', 'languages_of_instruction_examinations', 'graduation_project_theme', 'scientific_supervisor', 'graduation_project_mark', 'all_marks'])   

    subjects_list = []
    subjects_marks_begin_idx = text[text.find('Конец перечня / End of the list'):].find('\x0c') + text.find('Конец перечня / End of the list') + 2
    subjects_begin_idx = subjects_marks_begin_idx

    for i in range(3):
        
        marks_begin_idx = min(text[subjects_begin_idx:].find('удовлетворительно'), text[subjects_begin_idx:].find('хорошо'), text[subjects_begin_idx:].find('отлично'), text[subjects_begin_idx:].find('зачтено')) + subjects_begin_idx

        idx = marks_begin_idx
        count_n = 0
        while True:
            if text[idx] == '\n':
                count_n += 1
            idx -= 1
            if count_n >= 8:
                break
        subjects_end_idx = idx

        text1 = text[subjects_begin_idx: subjects_end_idx]

        for s in text1.split('\n\n'):
            subjects_list.append(s.split(' \n')[0])

        subjects_list1 = []
        for el in subjects_list:
            for symb in el:
                drop = True
                if rus(symb):
                    drop = False
                    break
            if not drop:
                subjects_list1.append(el)
        subjects_list = subjects_list1.copy()

        subjects_begin_idx = text[subjects_begin_idx:].find('\x0c') + subjects_begin_idx + 1

    marks_end_idx = text[subjects_marks_begin_idx:].find('Конец перечня') + subjects_marks_begin_idx
    marks_list = parse_marks(text[subjects_marks_begin_idx:marks_end_idx])

    marks_dict = dict()
    for i, key in enumerate(subjects_list):
        marks_dict[key] = marks_list[i]

    general_info.loc[len(general_info)] = [
        find_extract(text, 'Family name(s), given name(s) '),
        find_extract(text, 'Дата рождения (день/месяц/год)  '),
        find_extract(text, 'Access requirements '),
        find_extract(text, 'Admitted to '),
        find_extract(text, 'Graduated from '),
        find_extract(text, 'Level of education '),
        find_extract(text, 'Name of qualification '),
        find_extract(text, 'Main field of study '),
        find_extract(text, 'Academic programme title '),
        find_extract(text, 'Профессиональная траектория (профессиональные траектории): '),
        find_extract(text, 'Нормативный срок освоения ', without_new_str=True),
        find_extract(text, 'Форма обучения  ', without_new_str=True),
        find_extract(text, 'Language(s) of instruction / \nexaminations \n\n', without_new_str=True, until='/'),
        find_extract(text, 'Completion and defense \nof graduation project \n\n', until='\nНаучный руководитель').replace('\n', ''),
        find_extract(text, 'Научный руководитель: '),
        find_extract(text[text.find('ECTS \ngrade \n\nGrade') + len('ECTS \ngrade \n\nGrade'):], 'ECTS \ngrade \n\nGrade ')[4:],
        marks_dict
    ]



    return general_info

In [18]:
# pdf_path = '../data/files/РыбаковСергейВячеславович_Supl-1.pdf'
pdf_path = '../data/files/converted_from_docx.pdf'

result = parse_diploma_supplement(pdf_path)

In [20]:
result

,family_name,date_of_birth,access_requirements,admitted_to,graduated_from,level_of_education,name_of_qualification,main_field_of_study,academic_programme_title,professional_trajectory,official_length_of_programme,mode_of_study,languages_of_instruction_examinations,graduation_project_theme,scientific_supervisor,graduation_project_mark,all_marks
0,Рыбаков Сергей Вячеславович,05.07.2000,"Аттестат о среднем общем образовании, выданный...",Федеральное государственное бюджетное образова...,Федеральном государственном бюджетном образова...,бакалавриат,Бакалавр,01.03.02 Прикладная математика и информатика,"Прикладная математика, фундаментальная информа...",Математическое и программное обеспечение вычис...,4 года,очная,"English, Russian",Разработка масштабируемой архитектуры игрового...,кандидат технических наук И.С.Блеканов,отлично,"{'1С: Предприятие': 'отлично', 'Алгебра': 'удо..."


In [21]:
result.loc[0, 'all_marks']

{'1С: Предприятие': 'отлично',
 'Алгебра': 'удовлетворительно',
 'Аналитическая динамика управляемых систем': 'хорошо',
 'Английский язык': 'зачтено',
 'Архитектура вычислительных систем': 'зачтено',
 'Базы данных и сетевые технологии': 'зачтено',
 'Вариационное исчисление': 'удовлетворительно',
 'Введение в MATLAB': 'зачтено',
 'Геометрия': 'хорошо',
 'Дискретная математика': 'удовлетворительно',
 'Дифференциальные уравнения': 'удовлетворительно',
 'Информационные модели': 'зачтено',
 'История России (онлайн-курс)': 'зачтено',
 'Классификация документов': 'зачтено',
 'Концепции современного естествознания (на английском языке)': 'удовлетворительно',
 'Математический анализ I': 'удовлетворительно',
 'Математический анализ II. Теория функций комплексной переменной': 'зачтено',
 'Математическое моделирование': 'удовлетворительно',
 'Методы оптимизации': 'зачтено',
 'Моделирование социально-экономических систем': 'отлично',
 'Научно-исследовательская (производственная) практика': 'зачтено